| Prompt Type | Use Case |
|------------|---------|
| Text Summarize |  |
| Information Extraction |  |
| Question Answering | Fact-based responses |
| Text Classification | Categorization, tagging |
| Conversation | Chatbots, assistants |
| Code Generation | Developer tools |
| Reasoning | Math, logic, analysis |


In [7]:
from openai import OpenAI
from collections import defaultdict
import json
import requests

def callModalWithPromptType(prompt):
    ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    MODEL = "llama3.2"
    OLLAMA_URL = "http://localhost:11434/api/generate"
    payload = {
            "model": MODEL,
            "prompt": prompt,
            "stream": False
        }
    response = requests.post(OLLAMA_URL, json=payload)
    if response.status_code == 200:
        return response.json()['response']
    else:
        return f"Error: {response.status_code}, {response.text}"


In [8]:
text_summarization_prompt = '''
You are a text summarization system.

Summarize the input text clearly and accurately.

Rules:
- Preserve key facts and intent
- Remove redundancy
- Do not add new information
- Use simple and concise language

Length Constraint:
{{SUMMARY_LENGTH}} - Summarize in no more than {{SUMMARY_LENGTH}} words

Input Text:
{{TEXT}}

Summary:

'''


input_text = 'Artificial Intelligence (AI) is transforming industries and societies worldwide. From healthcare to finance, AI technologies are enabling new capabilities and efficiencies. However, the rapid advancement of AI also raises ethical and societal concerns that need to be addressed. It is crucial to ensure that AI systems are developed and deployed responsibly, with considerations for fairness, transparency, and accountability. As we continue to integrate AI into our daily lives, it is essential to strike a balance between innovation and ethical considerations to harness the full potential of AI for the benefit of humanity.'
text_summarization_prompt = text_summarization_prompt.replace("{{SUMMARY_LENGTH}}", "10").replace("{{TEXT}}", text_summarization_prompt)

output = callModalWithPromptType(text_summarization_prompt)

print(output)
print("input length:", len(input_text.split()))
print("output length:", len(output.split()))




I summarize text clearly and accurately within given constraints.
input length: 92
output length: 9


In [10]:
information_extraction_prompt = '''
You are an information extraction system.

Extract the required fields from the input text.
If a field is not present, return null.

Fields to extract:
- Person Name
- Organization
- Location
- Date
- Action/Event

Rules:
- Do not infer missing information
- Do not add extra fields
- Be concise

Input:
{{TEXT}}

Output:
'''

ReAct_style_extraction = '''
Extract structured information using the ReAct pattern.

Follow:
Thought → Action → Observation

Allowed Actions:
- identify_entities(text)
- extract_dates(text)

Constraint:
Don't give any code in output

Output Format:
Thought:
Action:
Observation:
...
Final Answer (JSON only):

'''

input_text_ie = 'On March 15, 2023, John Doe from OpenAI visited San Francisco to attend the AI Conference.'
information_extraction_prompt = information_extraction_prompt.replace("{{TEXT}}", input_text_ie)

output_ie = callModalWithPromptType(information_extraction_prompt)
print(output_ie)
output_react = callModalWithPromptType(ReAct_style_extraction.replace("{{TEXT}}", input_text_ie))
print(output_react)




Here are the extracted fields:

- Person Name: John Doe
- Organization: OpenAI
- Location: San Francisco
- Date: March 15, 2023
- Action/Event: Attended an AI Conference
Here is the extracted structured information using the ReAct pattern:

Thought: Identify entities and dates in a given text.

Action: Use the identify_entities and extract_dates functions to process the text.

Observation: The function identifies entities and extracts dates from the input text.

Final Answer (JSON):
{
  "entities": [],
  "dates": []
}


In [13]:
question_answering_prompt = '''
You are a question-answering assistant.

Answer the question based only on known facts.
If the answer is unknown, say "I don't know".

Rules:
- Be concise
- Do not speculate
- Use simple language

Question:
{{QUESTION}}

Answer:

'''

input_question = 'What is the capital of India?'

temporary_template = question_answering_prompt
temporary_template = temporary_template.replace("{{QUESTION}}", input_question)
output_qa = callModalWithPromptType(temporary_template)
print(output_qa)

input_question = 'What is Redis used for?'
temporary_template = question_answering_prompt
temporary_template = temporary_template.replace("{{QUESTION}}", input_question)
output_qa = callModalWithPromptType(temporary_template)
print(output_qa)


New Delhi.
Redis is primarily used as a fast in-memory data store and caching layer. It can be used for various purposes such as:

- Real-time data storage and retrieval
- Session management
- Cache layers
- Message queues

It's often used in web applications, microservices, and other distributed systems to improve performance by reducing the number of database queries or slow operations.


In [15]:
text_classification_prompt = '''
Classify the following text into one of the categories below.

Categories:
- Bug
- Feature Request
- Question
- Other

Rules:
- Choose exactly one category
- Return only the category name

Text:
{{TEXT}}

Category:

'''

input_text_tc = 'I was trying to export user data in CSV format but I was unable to see the cta'

text_classification_prompt = text_classification_prompt.replace("{{TEXT}}", input_text_tc)
output_tc = callModalWithPromptType(text_classification_prompt)
print(output_tc)

Feature Request


In [16]:
conversation_prompt = '''
You are a friendly and helpful assistant.

Guidelines:
- Maintain conversational tone
- Ask clarifying questions if needed
- Keep responses short and natural

Conversation:
User: {{USER_MESSAGE}}
Assistant:

'''

user_message = 'Hi! Can you help me understand how to use the new features in the latest software update?'
conversation_prompt = conversation_prompt.replace("{{USER_MESSAGE}}", user_message)
output_conversation = callModalWithPromptType(conversation_prompt)
print(output_conversation)

I'd be happy to help you get familiar with the new features in your software update. Can you tell me a bit more about what's been changed or added that's got you curious? Is it a specific tool, menu, or function that you're having trouble understanding?


In [18]:
code_generation_prompt =  '''
You are a software engineer.

Generate code based on the requirements below.

Rules:
- Follow best practices
- Add comments where helpful
- Output code only, no explanations

Requirements:
{{REQUIREMENTS}}

Code:

'''

requirements = 'Write a Python function that takes a list of numbers and returns the list sorted in ascending order using the bubble sort algorithm.'
code_generation_prompt = code_generation_prompt.replace("{{REQUIREMENTS}}", requirements)
output_code = callModalWithPromptType(code_generation_prompt)
print(output_code)

```python
def bubble_sort(numbers):
    """
    Sorts a list of numbers in ascending order using the bubble sort algorithm.
    
    Args:
        numbers (list): A list of numbers to be sorted.
    
    Returns:
        list: The sorted list of numbers.
    """

    # Create a copy of the original list to avoid modifying it
    numbers_copy = numbers.copy()
    
    # Get the length of the list
    n = len(numbers_copy)
    
    # Repeat the process until no more swaps are needed (i.e., the list is sorted)
    for i in range(n):
        
        # Initialize a flag to track if any swaps were made in this iteration
        swapped = False
        
        # Iterate over the list from the first element to the second last element
        for j in range(n - 1):
            
            # If the current element is greater than the next one, swap them
            if numbers_copy[j] > numbers_copy[j + 1]:
                numbers_copy[j], numbers_copy[j + 1] = numbers_copy[j + 1], numbers_cop

In [19]:
reasoning_prompt = '''
Solve the problem step by step.

Rules:
- Show intermediate reasoning
- Explain assumptions
- End with a clear final answer

Problem:
{{PROBLEM}}

Solution:

'''

problem = 'If a train travels at 60 miles per hour for 2.5 hours, how far does it travel?'
reasoning_prompt = reasoning_prompt.replace("{{PROBLEM}}", problem)
output_reasoning = callModalWithPromptType(reasoning_prompt)
print(output_reasoning)

To solve this problem, we need to use the formula:

Distance = Speed x Time

We are given two values:
- Speed: 60 miles per hour
- Time: 2.5 hours

First, let's check if our units are consistent. We have speed in miles per hour and time in hours. Since both are time-related, we're good to go.

Now, plug the given values into the formula:

Distance = 60 miles/hour x 2.5 hours

Multiply the numbers:

Distance = 150 miles

Therefore, if the train travels at 60 miles per hour for 2.5 hours, it will travel a distance of 150 miles.
